In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import yfinance as yf
import dash
from dash import dcc
from dash import html
import matplotlib.pyplot as plt

## importing data

In [2]:
df_data = pd.read_excel('data_norm.xlsx', header=[0,1], index_col=0)

In [3]:
data_sample = df_data.sample(n=25, random_state=2)

## Log returns

In [4]:
import log_return
log_re = log_return.log_returns(data_sample['Adj Close'])

In [5]:
from fetch_score import get_scores
esg_np, e_np, s_np, g_np = get_scores(data_sample)

## Covariance and inverse covariance matrix, and stocks mean return

In [6]:
from covariance import cov_matrix
from mean_ret import mean_return

cov_var = cov_matrix(log_re)
inv_cov = np.linalg.inv(cov_var)
mean_re = mean_return(log_re)

## Finding two optimal portfolios with different expected return

In [7]:
from ones import vector_ones
from mvp import mvp_func
from weights_mvp import mvp_weights

vector_one = vector_ones(data_sample)
a, b, c, d = mvp_func(inv_cov, mean_re, vector_one)
w1, w2 = mvp_weights(a, b, c, d, inv_cov, vector_one, mean_re, 0.10, 0.26)

## ESG insensitive investor

In [8]:
from alpha import alpha
from two_fun_theorem import w3

alpha_list = alpha([-3.5])
ef_weights, r, vol, esg_wa = w3(w1, w2, alpha_list, mean_re, cov_var, esg_np)

In [9]:
tickers_list = []
labels = data_sample.index
for i in range(len(ef_weights)):
    #assets = np.random.choice(list(labels), len(labels), replace=False)
    assets = list(labels)
    tickers_list.append(assets)
sr_t_list = list(labels)

MVP

In [10]:
from global_mvp import global_mvp
mvp_weight, mvp_er, mvp_vol, mvp_esg_score, mvp_e_score, mvp_s_score, mvp_g_score = global_mvp(ef_weights, vol, mean_re, cov_var, esg_np, e_np, s_np, g_np)

## ESG indicators

In [11]:
from esg_indicators import indicators
indicators(vector_one, inv_cov, mean_re, esg_np)

(0.23337328803346696, 0.4849035572299765)

## ESG sensitive investor 

In [12]:
from esg_investor_weights import calculate_esg_sensitive_investor
list_target_returns = np.arange(r.min(), r.max(), 0.0016).tolist()
len(list_target_returns)
esg_investor_weights, esg_portfolio_returns, esg_portfolio_vol, esg_investor_wa  = calculate_esg_sensitive_investor(mean_re, cov_var, esg_np, 0.46, list_target_returns, mvp_weight)

## Computing Sharpe Ratio

### Insenstive investor

In [13]:
from calculating_max_sr import calculate_max_sharpe_ratio
sr, w_opt, sharpe_exp, sharpe_vol, sr_esg_score, sr_e_score, sr_s_score, sr_g_score = calculate_max_sharpe_ratio(0.03, mean_re, cov_var, esg_np, e_np, s_np, g_np, mvp_weight, log_re.columns.tolist())

### Sensitive investor

In [14]:
from calculating_max_ESG_sr import calculate_max_ESG_sharpe_ratio
sr_esg, w_opt_esg, sharpe_exp_esg, sharpe_vol_esg, sr_esg_esg_score, sr_esg_e_score, sr_esg_s_score, sr_esg_g_score = calculate_max_ESG_sharpe_ratio(0.03, mean_re, cov_var, esg_np, e_np, s_np, g_np, mvp_weight, log_re.columns.tolist(), 0.46)

## Computing Sortino Ratio

### Insensitive investor

In [15]:
from calculating_max_sortino import calculate_sortino_ratio
optimal_weights, optimal_sortino_ratio, sortino_exp, sortino_vol, sortino_score, sortino_e_score, sortino_s_score, sortino_g_score = calculate_sortino_ratio(log_re, 0.03, log_re.columns.tolist(), mvp_weight, mean_re, cov_var, esg_np, e_np, s_np, g_np)

### Sensitive investor

In [16]:
from calculate_max_ESG_sortino import calculate_ESG_sortino_ratio
optimal_weights_esg, optimal_sortino_ratio_esg, sortino_exp_esg, sortino_vol_esg, sortino_score_esg, sortino_e_score_esg, sortino_s_score_esg, sortino_g_score_esg = calculate_ESG_sortino_ratio(log_re, 0.03, log_re.columns.tolist(), mvp_weight, mean_re, cov_var, esg_np, e_np, s_np, g_np, 0.46)

## Plotting the frontiers

In [17]:
from plot_ef import ef_plot
plot = ef_plot(tickers_list, sr_t_list, 
               ef_weights, vol, r, esg_wa, 
               esg_investor_weights, esg_portfolio_vol, esg_portfolio_returns, esg_investor_wa, 
               w_opt, sharpe_vol, sharpe_exp, sr, sr_esg_score,
               w_opt_esg, sharpe_vol_esg, sharpe_exp_esg, sr_esg, sr_esg_esg_score,
               mvp_weight, mvp_vol, mvp_er, mvp_esg_score, 
               optimal_weights, sortino_vol, sortino_exp, optimal_sortino_ratio, sortino_score, 
               optimal_weights_esg, sortino_vol_esg, sortino_exp_esg, optimal_sortino_ratio_esg, sortino_score_esg,
               ratio='Sortino Ratio')

plot.show()